In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya
LP = LilGuys.Plots

In [ ]:
using Printf

In [ ]:
import DataFrames: DataFrame, rename!

In [ ]:
models_dir = "/astro/dboyea/dwarfs/analysis/sculptor"

In [ ]:
sims_dir = "/astro/dboyea/dwarfs/simulations/sculptor"

In [ ]:
function load_profile(name) 
    path = joinpath(models_dir, "$name/profiles.hdf5")
    profiles = LilGuys.read_structs_from_hdf5(path, LilGuys.MassProfile3D)
    idx = parse.(Int, first.(profiles))
    return profiles[sortperm(idx)]
    
end

In [ ]:
t_max_host = 316 # computed with agama at peri=50.95 
M_max_host = 52.437
r_peri = 50.95
ρ_peri = M_max_host / (4π/3 * r_peri^3)

# Plotting functions

In [ ]:
function compare_densities(profiles)
    fig = Figure()
    ax = LP.rho_axis(fig[1, 1])
    ax.limits=(-2, 3, -12, 0)


    for i in eachindex(profiles)
        label, profs = profiles[i]
        profile = profs[1].second
        lines!(profile.log_r, log10.(profile.rho),
            linestyle=:dot,
            color=COLORS[i]
        )
    
        profile = profs[end].second
        lines!(profile.log_r, log10.(profile.rho), 
            color=COLORS[i],
            label=label
        )    
    end
    
    axislegend()

    fig
end

In [ ]:
function compare_vcirc(profiles; errskip=1)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel=LP.log_r_label,
        ylabel=L"$\log\,v_\textrm{circ}$ / km\,s$^{-1}$",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [prof.r_circ_max for (_, prof) in profs]
        y = [prof.v_circ_max for (_, prof) in profs]
        lines!(log10.(x), log10.(y*V2KMS), label=label)
    end

    axislegend(position=:lt)
    fig
end

In [ ]:
function compare_tcirc(profiles; errskip=1)
    xlims = (-2, 3)
    
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel="time / Gyr",
        ylabel=L"$\log\, t_\textrm{circ}$ / Gyr",
#        limits=(xlims[1], xlims[2], -0.1, 1.7),
        )
    pi = 1

    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        x = [parse(Float64, i) for (i, prof) in profs]
        y = [prof.r_circ_max / prof.v_circ_max for (_, prof) in profs]
        
        lines!(x, log10.(y*T2GYR), label=label)
    end
   
    fig
end

# Particle Number

In [ ]:
profiles = [
    "mean" => load_profile("1e6_V31_r3.2/orbit1"),
    "smallperi" => load_profile("1e6_V31_r3.2/orbit_smallperi"),
    "heavy" => load_profile("1e6_V40_r5.9/orbit_mean"),
    ];

In [ ]:
fig = compare_vcirc(profiles)
prof = profiles[1].second[1].second

lines!(prof.log_r, log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dot)
prof = profiles[1].second[end].second

lines!(prof.log_r, log10.(prof.v_circ * V2KMS), color=COLORS[1], linestyle=:dash)

ax = fig.content[1]
ax.limits = (-0.5, 1.6, 1.3, 1.65)
ax.xlabel = L"\log\,(r_\textrm{circ}\,/\,\textrm{kpc})"
fig

In [ ]:
compare_tcirc(profiles)

In [ ]:
compare_densities(profiles)